# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 10 2022 1:05PM,248283,10,Enova-10781,Emerginnova,Released
1,Oct 10 2022 1:05PM,248283,10,Enova-10782,Emerginnova,Released
2,Oct 10 2022 1:05PM,248283,10,Enova-10783,Emerginnova,Released
3,Oct 10 2022 1:05PM,248283,10,Enova-10785,Emerginnova,Released
4,Oct 10 2022 1:05PM,248283,10,DNMD-24626,Emerginnova,Released
5,Oct 10 2022 1:05PM,248283,10,Enova-10786,Emerginnova,Released
6,Oct 10 2022 1:05PM,248283,10,Eme-104924,Emerginnova,Released
7,Oct 10 2022 1:05PM,248283,10,1-10784,Emerginnova,Released
8,Oct 10 2022 12:00PM,248278,15,8628410,"Carwin Pharmaceutical Associates, LLC",Released
9,Oct 10 2022 11:49AM,248275,10,8628396-SD,Eywa Pharma Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
13,248262,Released,20
14,248274,Released,1
15,248275,Released,1
16,248278,Released,1
17,248283,Released,8


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
248262,NaN,NaN,NaN,20.0
248274,NaN,NaN,NaN,1.0
248275,NaN,NaN,NaN,1.0
248278,NaN,NaN,NaN,1.0
248283,NaN,NaN,NaN,8.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
248213,1.0,7.0,13.0,2.0
248228,0.0,0.0,0.0,11.0
248244,0.0,0.0,0.0,3.0
248245,0.0,17.0,16.0,6.0
248254,0.0,0.0,1.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
248213,1.0,7,13,2
248228,0.0,0,0,11
248244,0.0,0,0,3
248245,0.0,17,16,6
248254,0.0,0,1,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,248213,1.0,7,13,2
1,248228,0.0,0,0,11
2,248244,0.0,0,0,3
3,248245,0.0,17,16,6
4,248254,0.0,0,1,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,248213,1.0,7,13,2
1,248228,0.0,,,11
2,248244,0.0,,,3
3,248245,0.0,17,16,6
4,248254,0.0,,1,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id             int64
Cancelled    float64
Completed     object
Executing     object
Released       int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 10 2022 1:05PM,248283,10,Emerginnova
8,Oct 10 2022 12:00PM,248278,15,"Carwin Pharmaceutical Associates, LLC"
9,Oct 10 2022 11:49AM,248275,10,Eywa Pharma Inc.
10,Oct 10 2022 11:46AM,248274,10,"ClearSpec, LLC"
11,Oct 10 2022 10:56AM,248262,15,"Person & Covey, Inc."
31,Oct 10 2022 10:48AM,248260,15,"Brookfield Pharmaceuticals, LLC"
58,Oct 10 2022 10:38AM,248259,19,"GCH Granules USA, Inc."
59,Oct 10 2022 9:59AM,248254,10,"Senseonics, Incorporated"
61,Oct 10 2022 9:21AM,248245,10,Bio-PRF
100,Oct 10 2022 9:11AM,248244,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Cancelled,Completed,Executing,Released
0,Oct 10 2022 1:05PM,248283,10,Emerginnova,0.0,,,8
1,Oct 10 2022 12:00PM,248278,15,"Carwin Pharmaceutical Associates, LLC",0.0,,,1
2,Oct 10 2022 11:49AM,248275,10,Eywa Pharma Inc.,0.0,,,1
3,Oct 10 2022 11:46AM,248274,10,"ClearSpec, LLC",0.0,,,1
4,Oct 10 2022 10:56AM,248262,15,"Person & Covey, Inc.",0.0,,,20
5,Oct 10 2022 10:48AM,248260,15,"Brookfield Pharmaceuticals, LLC",0.0,,,27
6,Oct 10 2022 10:38AM,248259,19,"GCH Granules USA, Inc.",0.0,,,1
7,Oct 10 2022 9:59AM,248254,10,"Senseonics, Incorporated",0.0,,1,1
8,Oct 10 2022 9:21AM,248245,10,Bio-PRF,0.0,17,16,6
9,Oct 10 2022 9:11AM,248244,10,"Methapharm, Inc.",0.0,,,3


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 10 2022 1:05PM,248283,10,Emerginnova,8,,
1,Oct 10 2022 12:00PM,248278,15,"Carwin Pharmaceutical Associates, LLC",1,,
2,Oct 10 2022 11:49AM,248275,10,Eywa Pharma Inc.,1,,
3,Oct 10 2022 11:46AM,248274,10,"ClearSpec, LLC",1,,
4,Oct 10 2022 10:56AM,248262,15,"Person & Covey, Inc.",20,,
5,Oct 10 2022 10:48AM,248260,15,"Brookfield Pharmaceuticals, LLC",27,,
6,Oct 10 2022 10:38AM,248259,19,"GCH Granules USA, Inc.",1,,
7,Oct 10 2022 9:59AM,248254,10,"Senseonics, Incorporated",1,1,
8,Oct 10 2022 9:21AM,248245,10,Bio-PRF,6,16,17
9,Oct 10 2022 9:11AM,248244,10,"Methapharm, Inc.",3,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 10 2022 1:05PM,248283,10,Emerginnova,8,,
1,Oct 10 2022 12:00PM,248278,15,"Carwin Pharmaceutical Associates, LLC",1,,
2,Oct 10 2022 11:49AM,248275,10,Eywa Pharma Inc.,1,,
3,Oct 10 2022 11:46AM,248274,10,"ClearSpec, LLC",1,,
4,Oct 10 2022 10:56AM,248262,15,"Person & Covey, Inc.",20,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 10 2022 1:05PM,248283,10,Emerginnova,8,NaN,NaN
1,Oct 10 2022 12:00PM,248278,15,"Carwin Pharmaceutical Associates, LLC",1,NaN,NaN
2,Oct 10 2022 11:49AM,248275,10,Eywa Pharma Inc.,1,NaN,NaN
3,Oct 10 2022 11:46AM,248274,10,"ClearSpec, LLC",1,NaN,NaN
4,Oct 10 2022 10:56AM,248262,15,"Person & Covey, Inc.",20,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 10 2022 1:05PM,248283,10,Emerginnova,8,0.0,0.0
1,Oct 10 2022 12:00PM,248278,15,"Carwin Pharmaceutical Associates, LLC",1,0.0,0.0
2,Oct 10 2022 11:49AM,248275,10,Eywa Pharma Inc.,1,0.0,0.0
3,Oct 10 2022 11:46AM,248274,10,"ClearSpec, LLC",1,0.0,0.0
4,Oct 10 2022 10:56AM,248262,15,"Person & Covey, Inc.",20,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1986016,33,30.0,24.0
15,744800,48,0.0,0.0
19,248259,1,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1986016,33,30.0,24.0
1,15,744800,48,0.0,0.0
2,19,248259,1,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,33,30.0,24.0
1,15,48,0.0,0.0
2,19,1,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,33.0
1,15,Released,48.0
2,19,Released,1.0
3,10,Executing,30.0
4,15,Executing,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19
Status,,,
Completed,24.0,0.0,0.0
Executing,30.0,0.0,0.0
Released,33.0,48.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19
0,Completed,24.0,0.0,0.0
1,Executing,30.0,0.0,0.0
2,Released,33.0,48.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19
0,Completed,24.0,0.0,0.0
1,Executing,30.0,0.0,0.0
2,Released,33.0,48.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()